# Code to calculate tissue composition in GT tissue map, vs pix2pix, pyramid pix2pix, and I2SB:
### Since this is to calculate bulk tissue composition so that it is invariant to the slight pixel offsets vs GT, all you have to do is just sum up the masks and then represent the tissue composition as % by dividing by the total pixel area.

In [61]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2

In [7]:
# for i2sb, it is tmp, when finish sampling I2SB, do evaluation on all data:
gt_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE\classification_10172023"
p2p_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\pix2pix_sample1_only\classification_10162023"
# p_p2p_pth = # tbd
# i2sb_pth = # tbd

gt_images = [os.path.join(gt_pth,x) for x in os.listdir(gt_pth) if x.endswith(".png")]
p2p_images = [os.path.join(p2p_pth,x) for x in os.listdir(p2p_pth) if x.endswith(".png")]
# p_p2p_images = [os.path.join(p_p2p_pth,x) for x in os.listdir(p_p2p_pth) if x.endswith(".png")]
# i2sb_images = [os.path.join(i2sb_pth,x) for x in os.listdir(i2sb_pth) if x.endswith(".png")]

In [54]:
def return_tissue_composition(image_directory):
    pixel_value_counts = np.zeros(9, dtype=int)
    for filename in tqdm(image_directory,colour="red"):
        image = cv2.imread(filename, 0)
        unique, counts = np.unique(image, return_counts=True)
        pixel_value_counts[unique] += counts
    tissue_comp = [x/pixel_value_counts.sum() for x in pixel_value_counts]
    tissue_comp = [x*100 for x in tissue_comp]
    tissue_comp = tissue_comp[1:]
    return tissue_comp

In [55]:
gt_pixel_value_counts = return_tissue_composition(gt_images)
p2p_pixel_value_counts = return_tissue_composition(p2p_images)

100%|██████████| 9952/9952 [01:43<00:00, 95.81it/s] 


In [56]:
def find_MAE(list1,list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of equal length")
    absolute_differences = [abs(a - b) for a, b in zip(list1, list2)]
    mae = sum(absolute_differences) / len(absolute_differences)
    return mae

In [57]:
find_MAE(gt_pixel_value_counts,p2p_pixel_value_counts)

1.0859945196047474

### Note: 1~8 in order: islet duct vessels fat acini collagen whitespace nerves

In [58]:
p2p_pixel_value_counts

[0.2802698558549789,
 0.3572813567624598,
 1.2948856476418842,
 8.554825982096878,
 55.89510497556238,
 16.26526489135153,
 17.233209211343354,
 0.11915807938652408]

In [59]:
gt_pixel_value_counts

[1.145437676040306,
 0.7418683104193096,
 1.7018703019120685,
 6.052104913153449,
 56.5369057118701,
 14.424007881875973,
 19.155930092863716,
 0.2418751118650774]